# 데이터 전처리

---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd

---

# 1) 데이터 불러오기

In [3]:
%cd /content/gdrive/MyDrive/ColabNotebooks/[성록] 서울_시내버스_공모전/data/서울생활이동/

/content/gdrive/MyDrive/ColabNotebooks/[성록] 서울_시내버스_공모전/data/서울생활이동


In [4]:
# %ls

In [20]:
def count(dfs):
    return sum(map(lambda x: x.shape[0], dfs))

In [ ]:
def count(data):
    counts = [sum(map(lambda move: move.shape[0], moves)) for moves in data if moves is not None]
    print(f'총합: {sum(counts)}, count={counts}')

In [ ]:
year, month_range, time_range = 2022, range(1, 13), range(0, 24)

data = [[None] * (time_range.stop) for _ in range(month_range.stop + 1)]

for month in month_range:
    for time in time_range:
        df = pd.read_csv(f'생활이동_행정동_{year}{month:02d}/생활이동_행정동_{year}.{month:02d}_{time:02d}시.csv', encoding='cp949')
        data[month][time] = df

count(data)

In [16]:
year, month, time_range = 2022, 1, range(0, 24)

data = []
for time in time_range:
    df = pd.read_csv(f'생활이동_행정동_{year}{month:02d}/생활이동_행정동_{year}.{month:02d}_{time:02d}시.csv', encoding='cp949')
    data.append(df)

display(data[0].head())
display(count(data))

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202201,일,0,1101053,1101053,F,15,HH,10,4.04
1,202201,일,0,1101053,1101053,F,15,WH,10,4.04
2,202201,일,0,1101053,1101053,F,20,EH,10,*
3,202201,일,0,1101053,1101053,F,25,EE,10,*
4,202201,일,0,1101053,1101053,F,25,HH,10,*


115920824

In [55]:
data2 = data

# 2) 전처리

## a. 결측치 처리

- NaN 없음
- 이동인구(합)에서 `*`은 3명이하의 마스킹 값 -> 평균값(1.5)으로 대체

In [ ]:
# data.info()

In [17]:
for move in data:
    move['이동인구(합)'].replace('*', '1.5', inplace=True)
    move['이동인구(합)'] = move['이동인구(합)'].astype(float)

# 3) 데이터 줄이기

## a. 서울 -> 서울 데이터만

In [49]:
for i in range(len(data)):
    move = data[i]
    data[i] = move.loc[move['출발 행정동 코드'].between(1100000, 1199999) & move['도착 행정동 코드'].between(1100000, 1199999)]

print(count(data))

74461442


## a. 출발시간 컬럼 만들기

- 출발시간 = 도착시간 - 평균이동시간(분)//60
- if 출발시간 < 0: 출발시간 = 24 + 출발시간 and 요일 - 1

In [ ]:
for move in data:
    # 출발시간 = 도착시간 - 평균이동시간(분)//60
    move['출발시간'] = move['도착시간'] - move['평균 이동 시간(분)']//60

    # 출발시간 < 0 이면, 요일 -= 1
    day_to_num = {'월':0, '화':1, '수':2, '목':3, '금':4, '토':5, '일':6}
    num_to_day = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}

    move['요일2'] = move['요일'].map(day_to_num)
    move['요일2'] += move['출발시간'].map(lambda x: x if x < 0 else 0)
    move['요일'] = move['요일2'].map(num_to_day)
    move.drop(columns=['요일2'], inplace=True)

    # 출발시간 < 0 이면, 출발시간 += 24
    move['출발시간'] = move['출발시간'].map(lambda x: x + 24 if x < 0 else x)

In [51]:
data[0].loc[data[0]['출발시간'] < 0].shape

(0, 11)

# 3) 집계 데이터셋 만들기

- 시간대별 

In [53]:
groups = []
for move in data:
    groups.append(move.groupby(by='출발시간', as_index=False)['이동인구(합)'].sum())

move_time = pd.concat(groups, axis=0)

In [54]:
move_time

,출발시간,이동인구(합)
0,0,3526008.35
1,19,6.00
2,20,769.70
3,21,7385.09
4,22,49617.10
...,...,...
1,19,561.64
2,20,4567.38
3,21,35735.15
4,22,272065.84
